# Data Cleaning & Quality Checks


In this notebook I perform the data cleaning and basic data quality checks for all the datasets used in this project. The goal is to transform the raw GeoJSON files into clean GeoDataFrames that can be used for exploratory data analysis and spatial analysis.

This includes:
- Basic preprocessing (column selection, data types, CRS)
- Consistency and validity checks
- Spatial validity checks
- Fixing or removing invalid records

The cleaned datasets are saved into the `data/processed/` folder and will be used in the EDA notebooks.